In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
# tf.get_logger().setLevel('ERROR')

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        print(e)
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### utils

In [2]:
def class_mae(y_true, y_pred):
    return K.mean(
        K.abs(
            K.argmax(y_pred, axis=-1) - K.argmax(y_true, axis=-1)
        ),
        axis=-1
    )

In [3]:
# print(keras.__version__)
print(tf.__version__)

2.8.0


In [4]:
# print("Keras version :",keras.__version__)

# Convert model to new tensorflow

In [6]:
with open('json_config/CRNN_config.json') as json_file:
    json_config = json_file.read()

model = tf.compat.v1.keras.models.model_from_json(
    json_config,
    custom_objects={
        'class_mae': class_mae,
        'exp': K.exp,
    }
)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero1 (ZeroPadding2D)       (None, 1, 500, 201)       0         
                                                                 
 conv1 (Conv2D)              (None, 64, 498, 199)      640       
                                                                 
 conv2 (Conv2D)              (None, 32, 496, 197)      18464     
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 165, 65)       0         
                                                                 
 conv3 (Conv2D)              (None, 128, 163, 63)      36992     
                                                                 
 conv4 (Conv2D)              (None, 64, 161, 61)       73792     
                                                                 
 pool2 (MaxPooling2D)        (None, 64, 53, 20)       

In [7]:
model.load_weights("../CountNet/models/CRNN.h5")

# test

In [5]:
from time import time
RUN_FOR = 100

In [3]:
model = tf.keras.models.load_model("models/CRNN")

In [4]:
# tf random tensor
tf_random_tensor = tf.random.uniform((1, 1, 500, 201))
print("Input shape", tf_random_tensor.shape)
print("Output shape", model.predict(tf_random_tensor).shape)
tik = time()
for _ in range(RUN_FOR):
    model.predict(tf_random_tensor)
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Input shape (1, 1, 500, 201)
Output shape (1, 11)
Duration :7.59s


In [5]:
model = tf.keras.models.load_model("models/F-CRNN")

In [6]:
# tf random tensor
tf_random_tensor = tf.random.uniform((1, 500, 201))
print("Input shape", tf_random_tensor.shape)
print("Output shape", model.predict(tf_random_tensor).shape)
tik = time()
for _ in range(RUN_FOR):
    model.predict(tf_random_tensor)
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Input shape (1, 500, 201)
Output shape (1, 11)
Duration :6.74s


# ONNX

In [7]:
import onnxruntime as ort
import numpy as np

tf_random_tensor = tf.random.uniform((1,1, 500, 201))


ort_sess = ort.InferenceSession('onnx/CRNN_ONNX.onnx')
outputs = ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
# Print Result
result = outputs[0].argmax(axis=1)
print("Activations :", outputs[0][0])
print("label :",result[0])

tik = time()
for _ in range(RUN_FOR):
    ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Activations : [6.4035786e-12 3.2993048e-01 1.9193554e-03 1.1663444e-13 5.8555204e-20
 8.2125593e-20 2.3926074e-14 2.5142666e-09 3.6076106e-06 4.7517824e-03
 6.6339475e-01]
label : 10
Duration :2.29s


In [8]:
import onnxruntime as ort
import numpy as np

tf_random_tensor = tf.random.uniform((1, 500, 201))


ort_sess = ort.InferenceSession('onnx/F-CRNN_ONNX.onnx')
# ort_sess.get_inputs()[0].name
outputs = ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
# Print Result
result = outputs[0].argmax(axis=1)
print("Activations :", outputs[0][0])
print("label :",result[0])

tik = time()
for _ in range(RUN_FOR):
    ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Activations : [9.7518136e-09 3.2732623e-05 2.8245620e-10 3.4738325e-14 1.0145856e-15
 3.4067723e-14 9.9283914e-11 2.7187821e-07 6.6657034e-05 1.5107552e-02
 9.8479283e-01]
label : 10
Duration :0.06s
